# D’oh! Unevenly spaced time series analysis of The Simpsons in Pandas
#### [PyData Seattle 2017 tutorial](https://pydata.org/seattle2017/schedule/presentation/104/)
#### [Joe McCarthy](http://interrelativity.com/joe/), Data Scientist, [Indeed](https://www.indeed.com)



## Motivations

### Unevenly spaced opportunities to do unevenly spaced time series data analysis

<p><a href="http://pandas.pydata.org/"><img src="http://pandas.pydata.org/_static/pandas_logo.png" style="margin: 0px 0px 10px 20px; width: 150px; float: right;" title="pandas" alt="pandas_logo.png" /></a>I occasionally need to analyze gaps between events of interest, but not so often that I remember how to do it. I end up searching through my notebooks to remind myself which Pandas methods to use, and how to use them. I wanted to create a protypical example notebook that I can more easily reference .. and wanted to share it with others, in case they might also find it useful.</p>

### Data.world

<p><a href="https://data.world"><img src="https://d2ogkq1rg66kh0.cloudfront.net/site-resources/images/fb-image-share.7b15f964.jpg" style="margin: 0px 12px 5px 20px; width: 125px; float: right;" title="data.world" alt="data_world_logo.png" /></a>The last time I hunted down a notebook containing unevenly spaced time series analysis was around the time I first heard about the [data.world](https://data.world) platform for creating, sharing and collaborating on data sets .. I think of it as a GitHub for data.</p>

<p><a href="https://www.dataquest.io"><img src="http://data-science-hack.com/wp-content/uploads/2016/04/dataquest-io-1-1.png" style="margin: 0px 0px 5px 20px; width: 125px; float: right;" title="dataquest.io" alt="dataquest-io-logo.png" /></a>I was further inspired about using data.world after reading a great tutorial created by Josh Devlin at [Dataquest.io](https://www.dataquest.io) on [Turbocharge Your Data Acquisition using the data.world Python Library](https://www.dataquest.io/blog/datadotworld-python-tutorial/). This is also where I discovered the dataset about The Simpsons episodes, which I think is more entertaining and generally accessible than the datasets I typically work with at Indeed (although those are interesting, too).</p>

### PyData redemption

I gave a tutorial on [Python for Data Science](https://pydata.org/seattle2015/schedule/presentation/8/) at PyData Seattle 2015, wherein I tried to squeeze a 4-hour tutorial into a 2-hour slot. It didn't work out so well, though I did make the [Jupyter Notebook from the tutorial](http://nbviewer.jupyter.org/github/gumption/Python_for_Data_Science/blob/master/Python_for_Data_Science_all.ipynb) publicly available, so attendees could go back over some of the things we rushed through or didn't even get to. 

For PyData Seattle 2017, I decided to propose a topic that I estimated would take more like an hour, so that we can take a somewhat more leisurely approach .. and, with any luck, we'll be able to get to lunch before the other morning tutorials let out. 

And I will be making this notebook publicly available as well.

## Using Python and Pandas in a Jupyter Notebook

Among the prerequisites for this tutorial are to have Python, Pandas and Jupyter Notebook installed on your computer. In case anyone has a computer that does not yet meet these requirements, I recommend downloading and installing [Anaconda](https://www.continuum.io/downloads), an open data science platform for Python, which includes Pandas, Jupyter Notebook and a variety of other useful open source Python libraries. 

### Python 2 vs. Python 3

There are 2 major versions of Python in widespread use: [Python 2](https://docs.python.org/2/) and [Python 3](https://docs.python.org/3/). Python 3 has some features that are not backward compatible with Python 2, and some Python 2 libraries have not been updated to work with Python 3. I have been using Python 2, primarily because I use some of those Python 2[-only] libraries, but an increasing proportion of them are migrating to Python 3, and I anticipate shifting to Python 3 in the near future.

For more on the topic, I recommend a very well documented IPython Notebook, which includes numerous helpful examples and links, by [Sebastian Raschka](http://sebastianraschka.com/), [Key differences between Python 2.7.x and Python 3.x](http://nbviewer.ipython.org/github/rasbt/python_reference/blob/master/tutorials/key_differences_between_python_2_and_3.ipynb), the [Cheat Sheet: Writing Python 2-3 compatible code](http://python-future.org/compatible_idioms.html) by Ed Schofield ... or [googling Python 2 vs 3](https://www.google.com/q=python%202%20vs%203).

In order to make this notebook compatible with Python 2 *or* Python 3, we will import the [`print_function`]((https://docs.python.org/3/library/functions.html#print)) from the  [**`__future__`**](https://docs.python.org/2/library/__future__.html) module (in Python 2, `print` is a [statement](https://docs.python.org/2/reference/simple_stmts.html#print) not a function). We will also import the `division` module from the `future`, as I find [the use of `/` for "true division"](https://www.python.org/dev/peps/pep-0238/) - and the use of `//` for "floor division" - to be more aligned with my intuition.  

In [ ]:
from __future__ import print_function, division

We will also import - using the customary aliases - commonly used libraries in the python scientific stack, and specify that any plots will appear inline within the cell of this notebook (vs. popping up a separate window).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Using pandas

This tutorial is designed for people who are already familiar with the `pandas` data analysis library.

A thorough review of `pandas` concepts is beyond the scope of this session, but for participants who are not sufficiently familiar with `pandas`, I highly recommend the pandas [tutorial ("10 Minutes to pandas")](https://pandas.pydata.org/pandas-docs/stable/10min.html) and [Intro to Data Structures](https://pandas.pydata.org/pandas-docs/stable/dsintro.html).

Here are a few basic concepts:

* A [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) is a tabular (2-dimensional) data structure with 2 axes: rows (`axis=0`) and columns (`axis=1`)
* The columns in a `DataFrame` are one-dimensional arrays called [`Series`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html)
* Elements of a `DataFrame` can be accessed using labels, positions or boolean vectors
  * The labels for a `Series` or the rows of a `DataFrame` are accessible via their `index` attribute
  * The labels for each column in a `DataFrame` are accessible via its `columns` attribute
  * Individual labels can be specified using brackets or - in most cases - dotted notation
  * Positions can be specified using the [slice operators](https://docs.python.org/2/tutorial/introduction.html) for python sequences
  
Examples of these concepts will be highlighted when we first encounter them.

Other important `pandas` concepts and operations used throughout this tutorial include [grouping (split-apply-combine)](http://pandas.pydata.org/pandas-docs/stable/groupby.html) and [merging](http://pandas.pydata.org/pandas-docs/stable/merging.html)

## Using the data.world Python Library

In this section, I will borrow heavily - with permission - from Josh Devlin's tutorial, [Turbocharge Your Data Acquisition using the data.world Python library](https://www.dataquest.io/blog/datadotworld-python-tutorial/).

There are additional elements incorporated from the [Python SDK documentation](https://data.world/nrippner/explore-the-data-world-python-sdk/workspace/file?filename=ddw_SDK.ipynb) the [Data Wrangling tutorial](https://data.world/nrippner/python-data-wrangling-tutorial/workspace/file?filename=datadotworld_wrangling_tutorial.ipynb) at data.world.

### Installing the data.world library

The data.world Python SDK can be installed via the command line using the [`conda install` utility](https://conda.io/docs/using/pkgs.html#install-a-package-from-anaconda-org):

`conda install -c conda-forge datadotworld-py`

Alternately, you can use `pip` to install the SDK:

`pip install git+git://github.com/datadotworld/data.world-py.git`

Next, you need to retrieve and store your data.world API token.

* From your data.world account, go to [Settings > Advanced](https://data.world/settings/advanced) and get your API token (the image below is from Josh's tutorial):

<a href="https://data.world/settings/advanced"> <img src="https://www.dataquest.io/blog/images/data.world_tutorial/api_token.png" style="width: 600px;" title="data.world Advanced Settings" alt="data_world_api_token.png" /></a>

* If you installed the data.world Python SDK in a virtualenv or Conda env, activate that environment. 
* Run `dw configure` on the command line, which will prompt you for your token:
```
~ (datadotworld) $ dw configure
API token (obtained at: https://data.world/settings/advanced): _
```
When you enter your token, a `~/.dw/` directory will created in your home directory and your token will be stored in your `~/.dw/config` file.

Finally, if you used `pip install` above, you will need to run `pip install datadotworld[PANDAS]` on the command line.

### The Simpsons

For the benefit of anyone who is not familiar with The Simpsons, here is some context from the [Wikipedia page](https://en.wikipedia.org/wiki/The_Simpsons) for the television show:

<blockquote>
<p><a href="https://en.wikipedia.org/wiki/The_Simpsons"><img src="https://upload.wikimedia.org/wikipedia/en/thumb/0/0d/Simpsons_FamilyPicture.png/220px-Simpsons_FamilyPicture.png" style="margin: 0px 0px 10px 20px; width: 150px; float: right;" title="The Simpsons" alt="Simpsons_FamilyPicture.png" /></a>The Simpsons is an American animated sitcom created by Matt Groening for the Fox Broadcasting Company. The series is a satirical depiction of working-class life epitomized by the Simpson family, which consists of Homer, Marge, Bart, Lisa, and Maggie. The show is set in the fictional town of Springfield and parodies American culture, society, television, and the human condition.</p>

<p>...</p>

<p>Since its debut on December 17, 1989, 618 episodes of The Simpsons have been broadcast. Its 28th season began on September 25, 2016. It is the longest-running American sitcom and the longest-running American animated program, and, in 2009, it surpassed Gunsmoke as the longest-running American scripted primetime television series. The Simpsons Movie, a feature-length film, was released in theaters worldwide on July 27, 2007, and grossed over $527 million. On November 4, 2016, the series was renewed for a twenty-ninth and thirtieth season of 22 episodes each, extending the show to 2019.</p>
</blockquote>


### The Simpsons data set at data.world

Josh provides a brief history of this data set in [his tutorial](https://www.dataquest.io/blog/datadotworld-python-tutorial/):

> For this tutorial, we’ll be working with a data set of information on the TV show, [The Simpsons](https://en.wikipedia.org/wiki/The_Simpsons). The dataset was scraped by Tod Schenider for his post [The Simpsons by the Data](http://toddwschneider.com/posts/the-simpsons-by-the-data/), for which he made the scraper [available on GitHub](https://github.com/toddwschneider/flim-springfield). [Kaggle user William Cukierski used the scraper to upload the data set](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data), which was then [rehosted on data.world](https://data.world/data-society/the-simpsons-by-the-data).

[The data set page on data.world](https://data.world/data-society/the-simpsons-by-the-data) shows four CSV files in the data set:

* `simpsons_characters.csv`: Every character appearing in The Simpsons.
* `simpsons_episodes.csv`: Every episode of the The Simpsons.
* `simpsons_locations.csv`: Every location appearing in The Simpsons.
* `simpsons_script_lines.csv`: Most lines from most scripts of the Simpsons.

In _this_ tutorial, we'll only be using the data from `simpsons_episodes.csv` and `simpsons_script_lines.csv`.

### Using data.world’s Python library to explore the data

Continuing with [Josh's tutorial](https://www.dataquest.io/blog/datadotworld-python-tutorial/):

> First, let’s import the `datadotworld` library:

In [ ]:
import datadotworld as dw

> We’re going to use the `load_dataset()` function to take a look at the data. When we use `load_dataset()` for the first time, it:
> 
> * Downloads the data set from data.world and caches it in our `~/.dw/` directory
> * Returns a `LocalDataset` object representing the data set
> 
> Caching the data set locally is a really neat feature - it allows for quicker subsequent loading, lets you work on the data offline, ensures that your source data is the same each time you run your code, and in the future will support data set versioning. After the first time you call `load_dataset()` for a given dataset, it will load the dataset from the cached version. You can pass `True` to the optional `force_update` parameter if you wish to force a reload from the remote version and overwrite the changes.
> 
> `load_dataset()` has one required parameter, `dataset_key` which you can extract from the URL of the data set on data.world. As an example, our simpsons data set has the URL https://data.world/data-society/the-simpsons-by-the-data, which makes its ID `data-society/the-simpsons-by-the-data`.

In [ ]:
lds = dw.load_dataset('data-society/the-simpsons-by-the-data')  # , force_update=True)

After loading the dataset, we can verify that it has been locally cached.

In [ ]:
! ls -l ~/.dw/cache/data-society/the-simpsons-by-the-data/latest/data

### Getting to know our data

> To take a closer look at our `LocalDataset` object , we can use the `LocalDataset.describe()` method, which returns a JSON object.

In [ ]:
# We use pprint as it makes our output easier to read 
import pprint as pp

pp.pprint(lds.describe())

The JSON object has additional information since Josh created his tutorial, which has been inserted below:

> Our JSON object has 6 key/value pairs at the top level: 
* `description`
* `homepage` 
* `keywords` 
* `name` 
* `resouces` 
* `title`

> `resources` is a list that contains information about each file in our data.world data set:
* `name` 
* `format`
* `path`
* `mediatype`
* `bytes`

> Along with the `LocalDataset.describe()` function, there are three key attributes of our LocalDataset object which we can use to access the data itself:  
* `LocalDataset.dataframes`
* `LocalDataset.tables`
* `LocalDataset.raw_data`

> Each of these attributes work the same way, but return the data in a different format.

In [ ]:
for i in [lds.dataframes, lds.tables, lds.raw_data]:
    print(i, '\n')  # pprint does not workon lazy-loaded dicts

> `LocalDataset.dataframes` returns a dictionary of pandas DataFrame objects, where as `LocalDataset.tables` and `LocalDataset.raw_data` returns the data in dictionaries of Python lists and bytes format respectively. Lists can be useful if we don’t want to use pandas, and bytes is great if we have binary data like images or database files.
>
> Because of the power of the pandas library, let’s use `LocalDataset.dataframes` to explore and have some fun with our data!

## Exploratory analysis of The Simpsons data using `pandas`

data.world provides a number of ways of working with data, including their own version of SQL (dwSQL), SPARQL and an R SDK (see their [Intro to data.world Dataset tutorial](https://data.world/jonloyens/an-intro-to-dataworld-dataset)), but in this tutorial we will focus only on using the Python SDK and using the `pandas DataFrames` in their datasets.

We will create a `DataFrame` variable, `df`, to work with data from the `simpsons_script_lines` CSV file.

In [ ]:
df = lds.dataframes['simpsons_script_lines']

The warnings show that we have some columns with mixed types in this `DataFrame`.

```
/Users/joem/anaconda/lib/python2.7/site-packages/datadotworld/models/dataset.py:192: UserWarning: Unable to set data frame dtypes automatically using simpsons_script_lines schema. Data types may need to be adjusted manually. Error: Integer column has NA values in column 7
  'Error: {}'.format(resource_name, e))
/Users/joem/anaconda/lib/python2.7/site-packages/datadotworld/util.py:136: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return self._loader_func()
```

We can use the [`pandas.DataFrame.info()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html) function to get more information on the contents of these columns.

In [ ]:
df.info()

9 columns have type `object`, which is usually an indicator that the column contains a mix of data types. 

Sometimes this happens when columns include null (`NaN`) values, so let's show the number of null values along with the number of data types, and include the number of unique values in each column. 

In [ ]:
for i, column_label in enumerate(df.columns):
    print('{:2d}: {:20} {:6d} unique values, {:6d} null values, {:2d} data type(s)'.format(
        i,
        column_label + ':', 
        df[column_label].nunique(),
        df[column_label].isnull().sum(),
        len(df[column_label].apply(lambda x: type(x)).value_counts())))

We can also see what kinds of values are stored in the columns that have mixed data types.

In [ ]:
for i, column_label in enumerate(df.columns):
    if len(df[column_label].apply(lambda x: type(x)).value_counts()) > 1:
        print(i, column_label)
        print(df[column_label].apply(lambda x: type(x)).value_counts())
        print()

## Data Preparation & Cleaning

There are a number of columns which have more than one data type, e.g., `str` and `float`. There are some other data quality issues in the `DataFrame` created from `simpsons_script_lines.csv`.

In [ ]:
df.head()

Another [notebook](pydata-tutorial-simpsons-data-cleaning.ipynb) in the repository contains a more detailed data analysis and cleaning of the `DataFrame`.

For now, we'll just import a module with some utility functions and run a function to do the cleaning.

If there is time - and interest - toward the end of the tutorial, we can review that notebook.

In [ ]:
import pydata_simpsons

df = pydata_simpsons.clean_simpsons_script_lines(df)

In [ ]:
df.head()

We will also use data from the `simpsons_episodes.csv` file to map episodes to seasons, so we will create a separate `DataFrame` for working with the episodes data.

In [ ]:
df_episodes = lds.dataframes['simpsons_episodes']
len(df_episodes)

In [ ]:
df_episodes.info()

In [ ]:
df_episodes.head()

The only field we need from the episodes data is `season`, and we'll need the `id` field to map to the `episode_id` field in the `simpsons_script_lines` data, so we'll [merge](http://pandas.pydata.org/pandas-docs/stable/merging.html) those fields from `df_episodes` with `df` (using [`pandas.DataFrame.merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)), and then drop the extraneous `id` column (`axis=1`) from `df_episodes` (using [`pandas.DataFrame.drop()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)).

In [ ]:
df = df.merge(
    df_episodes[['id', 'season']],
    left_on='episode_id',
    right_on='id',
    how='left',
    suffixes=['', '_y']).drop('id_y', axis=1)

The new `season` field will be in the rightmost column of the `DataFrame`, but I would like to see it appear right after the `id` field, so will re-order the columns.

In [ ]:
df = df[[u'id', u'season', u'episode_id', u'number', u'raw_text', u'timestamp_in_ms',
       u'speaking_line', u'character_id', u'location_id',
       u'raw_character_text', u'raw_location_text', u'spoken_words',
       u'normalized_text', u'word_count']]

In [ ]:
df.head()

## Time Series Analysis

From [Wikipedia](https://en.wikipedia.org/wiki/Time_series):

> A **time series** is a series of data points indexed (or listed or graphed) in time order. Most commonly, a time series is a sequence taken at successive equally spaced points in time. Thus it is a sequence of discrete-time data. Examples of time series are heights of ocean tides, counts of sunspots, and the daily closing value of the Dow Jones Industrial Average.
>
> **Time series *analysis*** comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data.

### Time Series data from The Simpsons

In [The Simpsons dataset on data.world](https://data.world/data-society/the-simpsons-by-the-data), we will use two `DataFrames`, each loaded from its corresponding CSV file.

Each data point (row) in the DataFrame we loaded from **`simpsons_script_lines.csv`** represents a distinct script line, indexed by an **`id`** field that indicates the order in which the line occured.

Other fields in this `DataFrame` include
* **`episode_id`**: an identifier for each episode
* **`number`**: an identifier for the position of each line within an episode
* **`raw_text`**: the raw text extracted from ???
* **`timestamp_in_ms`**: the number of milliseconds since the start of the season when the line occurred
* **`speaking_line`**: a boolean indicator of whether the line was a speaking line
* **`character_id`**: a unique identifier for each character
* **`location_id`**: a unique identifier for each location
* **`raw_character_text`**: the full text (name) of the character
* **`raw_location_text`**: the raw text (name) of the location
* **`spoken_words`**: the text spoken in the line (if any)
* **`normalized_text`**: a lowercase version of the `spoken_words`, with most punctuation removed
* **`word_count`**: the number of space-delimited tokens in `normalized_text`

Each data point in the `DataFrame` loaded from **`simpsons_episodes.csv`** represents a distinct episode, which is also indexed by an **`id`** field that indicates the order in which the episode occured. This `id` field maps to the `episode_id` in `simpsons_script_lines.csv`. The only other field from this `DataFrame` we will use is **`season`**, which is a number indicating the season in which the episode originally aired.

Note that although there are 600 `id` values in `simpsons_episodes.csv`, there are only 564 `episode_id` values in `simpsons_script_lines.csv`.

In [ ]:
df_episodes.id.nunique(), df.episode_id.nunique(), len(df_episodes[~df_episodes.id.isin(df.episode_id)])

In [ ]:
df_episodes[~df_episodes.id.isin(df.episode_id)]

### Exercise Set A

Now that we have our base datasets, we can do some time series analyses on the data. 

#### Exercise A1: What are the minimum, maximum and mean number of script lines per episode?

You may find it helpful to use the following `pandas` functions:
* [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)
* [`pandas.Series.nunique()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html)
* [`pandas.Series.min()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.min.html), [`pandas.Series.max()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.max.html), [`pandas.Series.mean()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.mean.html) .. and/or [`pandas.core.groupby.DataFrameGroupBy.agg()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)

In [ ]:
# your code here

#### Exercise A2: Show (plot) the distribution of the number of script lines per episode.

You may find it helpful to use the [`pandas.DataFrame.hist()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.hist.html) function.

In [ ]:
# your code here

#### Exercise A3: What are the minimum, maximum and mean number of episodes per season?

In [ ]:
# your code here

#### Exercise A4: Show (plot) the distribution of the number of episodes per season.

In [ ]:
# your code here

#### Exercise A5: What are the minimum, maximun and mean number of characters appearing per episode?

In [ ]:
# your code here

#### Exercise A6: Show (plot) the distribution of the number of characters per episode.

In [ ]:
# your code here

### Exercise Set B

Some of the interesting questions we might ask about The Simpsons are focused on the characters.

In order to simpify analysis of The Simpsons characters, it will be useful to construct a new `DataFrame` containing data associated with each character (vs. data associated with each episode or script line, although we will use data from both sources in constructing the new `DataFrame`).

The following exercises will focus primarily on using some standard `pandas` functions, and set the stage for doing some additional time series analysis.

#### Exercise B1: Determine the number of lines associated with each character

_(We will do this together)_

First, it is important to note that we have two fields in the `DataFrame` extracted from `simpsons_script_lilnes.csv` that are associated with characters:
* `character_id`: a unique identifier for each character
* `raw_character_text`: the full text (name) associated in the script with each appearance of a character

And it is further important to note that there are more distinct `raw_character_text` values than there are distinct `character_id` values. 

In [ ]:
df.raw_character_text.nunique(), df.character_id.nunique()

If we group the script lines by `character_id` (using [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)) and then gather the distinct `raw_character_text` values associated with each `character_id` (using [`pandas.Series.unique()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html)), the first few rows indicate why there are more `raw_character_text` values than `character_id` values.

A given character can have different personae within the context of a show (e.g., "Marge Simpson" and "Young Marge").

In [ ]:
df.groupby('character_id').raw_character_text.unique().head()

If we group by both `character_id` and `raw_character_text`, and then find the number of rows in each group (using [`pandas.core.groupby.GroupBy.size()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.size.html)), and then convert the resulting [`MultiIndex`](https://pandas.pydata.org/pandas-docs/stable/advanced.html) object back into a `DataFrame` (using [`pandas.DataFrame.reset_index()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html)), we can create a new DataFrame with a row for each `raw_character_text` value, wherein each row has 
* `character_id`: a non-unique value which may be shared across other `raw_character_text` rows
* `raw_character_text`: a unique full text (name) for the character
* a column indicating the number of script lines associated with that `raw_character_text` (labeled `0`)

In [ ]:
df.groupby(['character_id', 'raw_character_text']).size().reset_index().head(10)

Alternatively, we could use [`pandas.Series.nunique()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html) to count the number of unique `id` values within each group.

In [ ]:
df.groupby(['character_id', 'raw_character_text'])['id'].nunique().reset_index().head(10)

We can rename the column showing the number of script lines (using [`pandas.DataFrame.rename()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html)), and assign the resulting `DataFrame` to a new variable, `df_characters`.

In [ ]:
df_characters = df.groupby(['character_id', 'raw_character_text'])['id'].nunique().reset_index().rename(
    columns={'id': 'num_lines'})

In [ ]:
df_characters.head(10)

We can sort the characters based on the number of lines they have (using [`pandas.DataFrame.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html).

In [ ]:
df_characters.sort_values('num_lines', ascending=False).head(10)

We can exclude the null character by restricting the rows to those for which `character_id>0` or those for which `raw_character_text!=''`:

In [ ]:
df_characters[df_characters.character_id>0].sort_values('num_lines', ascending=False).head(10)

#### Exercise B2: Determine the number of episodes in which each character appears.

More specifically: add a new column to `df_characters`, named `num_episodes`, containing a count of the number of `episode_id` values associated with each `raw_character_text` value.

It may be useful to break this down into two steps:
* Create a new `DataFrame` containing the number of `episode_id` values associated with each `raw_character_text` value (hint: you might use  [`pandas.Series.nunique()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html) again)
* [Merge](http://pandas.pydata.org/pandas-docs/stable/merging.html) that `DataFrame` with the existing `df_characters` (using [`pandas.DataFrame.merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html))

In [ ]:
# your code here

#### Exercise B3: Determine the first and last episodes in which each character appears

More specifically, add the following columns to `df_characters`:
* `first_episode_id`: the first (smallest) `episode_id` in which the associated `raw_character_text` value appears
* `last_episode_id`: the last (largest) `episode_id` in which the associated `raw_character_text` value appears

You can either follow the pattern in the previous exercise - twice - or combine the two into a single grouping (using the [`pandas.core.groupby.DataFrameGroupBy.agg()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)) before merging.

In [ ]:
# your code here

#### Exercise B4: Determine the number of seasons, the first season and the last season in which a character appears

More specifically, add the following columns to `df_characters`:
* `num_seasons`: the number of distinct `seasons` in which the associated `raw_character_text` appears
* `first_season`: the first `season` in which the associated `raw_character_text` value appears
* `last_episode_id`: the last `season` in which the associated `raw_character_text` value appears

You should be able to modify the code you wrote for Exercises 2 and 3 above to address this task.

In [ ]:
# your code here

Your `DataFrame` should look similar to the one below.

In [ ]:
df_characters = pydata_simpsons.create_simpsons_characters_dataframe(df)

In [ ]:
df_characters.head(10)

In [ ]:
df.head()

### Exercise Set C

Now that we have `df_characters` constructed, we can do some additional analysis (more easily).

#### Exercise C1: What are the minimum, maximum and mean number of script lines per character?

In [ ]:
# your code here

## Unevenly spaced time series analysis

As noted earlier (from the Wikipedia article on Time series):

> Most commonly, a time series is a sequence taken at successive equally spaced points in time

Unevenly spaced time series analysis involves data points that are spaced over a sequence of unequal time intervals. The sizes of the intervals themselves are often a focus of such analysis.

Examples of such analyses I've done at Indeed include
* identifying "repeat business" from Indeed Hire clients, based on different definitions of "repeat", by analyzing the intervals between the dates on which new job requisitions from the same client were created
* determining how much time Indeed Hire recruiters spend reviewing candidate resumes and cover letters, by analyzing the intervals between candidate profile dispositions

Within the context of this tutorial, we'll restrict our focus to unevenly spaced time series analysis of The Simpsons.

### Identify scene changes (an unevenly spaced time series analysis)

One potential indicator of scene changes is changes of location.

One way to identify changes of location is to create a new column with the location id of the previous script line and then look for rows in which the previous location id is different from the [current] location id.

It is important to remember that there is noise in the data, including script lines for which no location information is available. 

In [ ]:
(len(df[df.location_id==0]), 
 len(df[df.raw_location_text=='']), 
 len(df[(df.location_id==0) & (df.raw_location_text=='')]))

In [ ]:
df[df.location_id==0].head()

As with character data, some `location_id` values are associated with more than one `raw_location_text` value.

In [ ]:
(df.location_id.nunique(), 
 df.raw_location_text.nunique(), 
 df.raw_location_text.str.lower().nunique())

In [ ]:
df[df.location_id>0].raw_location_text.value_counts().head()

In [ ]:
df[df.location_id>0].raw_location_text.value_counts().tail()

In the context of this tutorial, we will simply forge ahead, noting that the results below are, at best, approximate.

### Exercise Set D

#### Exercise D1: Add a column, `prev_location_id`, to contain the previous  `location_id` for each script line within a given season

The [`pandas.DataFrame.shift()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shift.html) function can be used to return the rows of a `DataFrame` shifted by some offset (the default offset is 1 row). Positive offsets can be interpreted as shifting "down", which allows us to easily access previous rows.

In [ ]:
df.location_id.head()

In [ ]:
df.location_id.shift().head()

We can also use negative offset values to shift "up" for next rows.

In [ ]:
df.location_id.shift(-1).head()

We'll want to first group by episode.

In [ ]:
df.groupby(['episode_id']).location_id.shift().head()

And to keep things consistent with the `location_id` field, we can substitute zero for null (`NaN`) values.

In [ ]:
df.groupby('episode_id').location_id.shift().fillna(0).astype('int64').head()

In [ ]:
df.loc[:, 'prev_location_id'] = df.groupby('episode_id').location_id.shift().fillna(0).astype('int64')

In [ ]:
df[['location_id', 'prev_location_id']].head()

#### Exercise D2: How many scene changes are there across all episodes?

If we conveniently ignore the fact that we have rows with missing locations (where `location_id==0`), we can compute this by simply counting the number of rows where the `location_id` and `prev_location_id` differ.

In [ ]:
len(df[df.location_id != df.prev_location_id])

Most script lines take place within the same scene (location).

In [ ]:
len(df[df.location_id == df.prev_location_id])